<a href="https://colab.research.google.com/github/ipmob/NITD-machine-Learning-challange/blob/master/Final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install -i https://test.pypi.org/simple/ supportlib



Looking in indexes: https://test.pypi.org/simple/


In [0]:
import supportlib.gettingdata as getdata
getdata.kaggle()

Saving kaggle.json to kaggle.json


In [0]:
!pip install -q kaggle

In [0]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

In [0]:
!kaggle datasets download -d thanakomsn/glove6b300dtxt

 98% 377M/386M [00:09<00:00, 34.0MB/s]
100% 386M/386M [00:09<00:00, 41.3MB/s]


In [0]:

getdata.zipextract('/content/glove6b300dtxt.zip')
#!pwd

In [0]:
!git clone https://github.com/ipmob/NITD-machine-Learning-challange

Cloning into 'NITD-machine-Learning-challange'...
remote: Enumerating objects: 12, done.
remote: Counting objects: 100% (12/12), done.
remote: Compressing objects: 100% (11/11), done.
remote: Total 12 (delta 1), reused 8 (delta 0), pack-reused 0
Unpacking objects: 100% (12/12), done.


In [0]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

import re
import keras

import nltk 
nltk.download('punkt')

nltk.download('stopwords')
from nltk.corpus import stopwords
stopwords = set(stopwords.words('english'))

Using TensorFlow backend.


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [0]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

import re
import keras

import nltk 
nltk.download('punkt')

nltk.download('stopwords')
from nltk.corpus import stopwords
stopwords = set(stopwords.words('english'))

Using TensorFlow backend.


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [0]:
data = pd.read_csv('/content/NITD-machine-Learning-challange/data/train_file.csv')
test = pd.read_csv('/content/NITD-machine-Learning-challange/data/test_file.csv')
sample = pd.read_csv('/content/NITD-machine-Learning-challange/data/results_file.csv')

data.loc[data.Subjects.isnull(), 'Subjects'] = ''
test.loc[test.Subjects.isnull(), 'Subjects'] = ''

data['PublicationYear'] = data['PublicationYear'].astype('str').map(lambda x: " ".join(re.findall('\d{4}', x)))
test['PublicationYear'] = test['PublicationYear'].astype('str').map(lambda x: " ".join(re.findall('\d{4}', x)))
test = test.fillna({'Subjects': "", "Title": "", "Publisher": ""})
data = data.fillna({"PublicationYear":"","Publisher":""})

data['combined'] = data['Subjects'] + " " + data['Title'] + " " + data['PublicationYear'].astype('str') + " " + data['Publisher'] + " " + data['Checkouts'].astype('str')
test['combined'] = test['Subjects'] + " " + test['Title'] + " " + test['PublicationYear'].astype('str') + " " + test['Publisher'] + " " + data['Checkouts'].astype('str')

# data['combined'] = data.Subjects.str.cat(' ' + data.Title)
# test['combined'] = test_data['Subjects']

# data['combined'] = td2['sub']

dataMatType = data.MaterialType
data.drop(labels = ['MaterialType'], axis = 1, inplace = True)

data.combined = data.combined.apply(preprocess_text)
test.combined = test.combined.apply(preprocess_text)
'''
data.Subjects = data.Subjects.apply(preprocess_text)
data.Title = data.Title.apply(preprocess_text)
test.Subjects = test.Subjects.apply(preprocess_text)
test.Title = test.Title.apply(preprocess_text)
'''
flag = -1
labelDict = {}

for key in dataMatType.value_counts().keys():
  flag += 1
  labelDict[key] = flag
  
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import numpy as np

maxlen = 25                            

max_words = 1000                                    

tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(data.combined)

dataSequences = tokenizer.texts_to_sequences(data.combined)
testSequences = tokenizer.texts_to_sequences(test.combined)

data_word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(data_word_index))

Found 37290 unique tokens.


In [0]:
from keras import preprocessing
x_train = preprocessing.sequence.pad_sequences(dataSequences, maxlen = 25)
x_test = preprocessing.sequence.pad_sequences(testSequences, maxlen = 25)

train_label = dataMatType.map(labelDict)

from keras.utils import to_categorical
labelDataBinary = to_categorical(train_label)

from sklearn.utils import class_weight
class_weights = class_weight.compute_class_weight('balanced', np.unique(dataMatType), dataMatType)

In [0]:
import os
glove_dir = '/content'

embeddings_index = {}
f = open(os.path.join(glove_dir, 'glove.6B.300d.txt'))
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

print('Found %s word vectors.' % len(embeddings_index))

embedding_dim = 300

embedding_matrix = np.zeros((max_words, embedding_dim))
for word, i in data_word_index.items():
    if i < max_words:
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None:
            embedding_matrix[i] = embedding_vector

Found 400000 word vectors.


In [0]:
reverseLabelDict = {}

for key, value in labelDict.items():
  reverseLabelDict[value] = key
def predict(model, x_test):
  predictions = model.predict(x_test)
  predictions = pd.Series([np.argmax(i) for i in predictions], index = test.index)
  predictions = predictions.map(reverseLabelDict)
  return predictions

NameError: ignored

In [0]:
from keras.layers import LSTM
from keras.models import Sequential
from keras.layers import Dense, Flatten, Embedding, SimpleRNN, Conv1D, MaxPooling1D
from keras.layers import Dropout
from keras.layers import Dense

model1 = Sequential()
model1.add(Embedding(1200, 1200, input_length=25))

'''model.add(Conv1D(256, 5, activation='relu'))
model.add(MaxPooling1D(pool_size=4))
'''


model1.add(Conv1D(128, 3,  activation='elu'))
model1.add(Dropout(0.6))

model1.add(LSTM(400, recurrent_dropout = 0.6, return_sequences = True))
model1.add(Dropout(0.4))

model1.add(LSTM(400, recurrent_dropout = 0.5))
model1.add(Dropout(0.2))

model1.add(Dense(8, activation='softmax'))

model1.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['acc'])
history = model1.fit(x_train, labelDataBinary,
                    epochs=16,
                    validation_split=0,
                    class_weight = class_weights)

model1.save('/content/m1.h5')
#0.86623 
#0.87633

predic1 = pd.Series(predict(model1, x_test))
submission = pd.concat([test.ID.astype(np.int), predic1], axis = 1)
submission.columns = ['ID' , 'MaterialType']
submission.to_csv('/content/submition_model1.1_add.csv', index = False, header = True)  


Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.cast instead.
Epoch 1/16
31653/31653 [==============================] - 124s 4ms/step - loss: 0.4978 - acc: 0.8612
Epoch 2/16
31653/31653 [==============================] - 121s 4ms/step - loss: 0.4156 - acc: 0.8789
Epoch 3/16
31653/31653 [==============================] - 121s 4ms/step - loss: 0.4048 - acc: 0.8834
Epoch 4/16
31653/31653 [==============================] - 120s 4ms/step - loss: 0.4032 - acc: 0.8848
Epoch 5/16
31653/31653 [==============================] - 120s 4ms/step - loss: 0.4009 - acc: 0.8853
Epoch 6/16
31653/31653 [==============================] - 124s 4ms/step - loss: 0.4006 - acc: 0.8865
Epoch 7/16
31653/31653 [==============================] - 121s 4ms/step - loss: 0.3965 - acc: 0.8874
Epoch 8/16
31653/31653 [==============================] - 

In [0]:
# from keras.layers import CuDNNLSTM,CuDNNGRU
# from keras.models import Sequential
# from keras.layers import Dense, Flatten, Embedding, SimpleRNN, Conv1D, MaxPooling1D
# from keras.layers import Dropout
# from keras.layers import Dense
# from keras import optimizers,callbacks
# import keras
# model2 = Sequential()
# model2.add(Embedding(1000, 1000 ,input_length=25))



# model2.add(Conv1D(256, 5, activation=r'elu'))
# model2.add(Dropout(0.6))
# model2.add(Conv1D(512, 5, activation='elu'))
# model2.add(Dropout(0.3))
# model2.add(Conv1D(1024, 5, activation='elu'))
# model2.add(Dropout(0.5))

# model2.add(Flatten())

# model2.add(Dense(400, activation='elu'))
# model2.add(Dense(100, activation='elu'))
# model2.add(Dense(8, activation='softmax'))

# #adam = optimizers.adam(lr = 0.0005)
# adam = optimizers.adam(lr = 0.0026)
# reduce_lr = callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=2)
# model2.compile(optimizer=adam,
#               loss='categorical_crossentropy',
#               metrics=['acc'])
# history = model2.fit(x_train, labelDataBinary,
#                     epochs=16,
#                     validation_split=0,
#                     class_weight = class_weights,
#                    callbacks=[reduce_lr])

# model2.save('/content/m2.h5')
# #0.85621
# predic2 = pd.Series(predict(model2, x_test))
# submission = pd.concat([test.ID.astype(np.int), predic2], axis = 1)
# submission.columns = ['ID' , 'MaterialType']
# submission.to_csv('/content/submition_model2_add.csv', index = False, header = True)  


Epoch 1/16
31653/31653 [==============================] - 36s 1ms/step - loss: 5.2104 - acc: 0.6757
Epoch 2/16
  160/31653 [..............................] - ETA: 36s - loss: 4.5332 - acc: 0.7188

/usr/local/lib/python3.6/dist-packages/keras/callbacks.py:1109: RuntimeWarning: Reduce LR on plateau conditioned on metric `val_loss` which is not available. Available metrics are: loss,acc,lr
  (self.monitor, ','.join(list(logs.keys()))), RuntimeWarning


31653/31653 [==============================] - 34s 1ms/step - loss: 5.0697 - acc: 0.6855
Epoch 3/16
31653/31653 [==============================] - 34s 1ms/step - loss: 5.0712 - acc: 0.6854
Epoch 4/16
27040/31653 [========================>.....] - ETA: 4s - loss: 5.0816 - acc: 0.6847

KeyboardInterrupt: ignored

In [0]:
from keras.layers import CuDNNLSTM,CuDNNGRU
from keras.models import Sequential
from keras.layers import Dense, Flatten, Embedding, SimpleRNN, Conv1D, MaxPooling1D
from keras.layers import Dropout
from keras.layers import Dense
from keras import optimizers,callbacks
import keras
model3 = Sequential()
model3.add(Embedding(1000, 1000 ,input_length=25))


model3.add(CuDNNGRU(600, return_sequences = True))
model3.add(Dropout(0.7))

model3.add(CuDNNGRU(600, return_sequences = True))
model3.add(Dropout(0.5))

model3.add(CuDNNGRU(800))
model3.add(Dropout(0.4))

model3.add(Dense(200, activation='elu'))
model3.add(Dense(50, activation='elu'))
model3.add(Dense(8, activation='softmax'))

adam = optimizers.adam(lr = 0.0014)
reduce_lr = callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=2)
model3.compile(optimizer=adam,
              loss='categorical_crossentropy',
              metrics=['acc'])
history = model3.fit(x_train, labelDataBinary,
                    epochs=16,
                    validation_split=0,
                    class_weight = class_weights,
                   callbacks=[reduce_lr])

model3.save('/content/m3.h5')
#0.87688 
predic3 = pd.Series(predict(model3, x_test))
submission = pd.concat([test.ID.astype(np.int), predic3], axis = 1)
submission.columns = ['ID' , 'MaterialType']
submission.to_csv('/content/submition_model3_add_2.csv', index = False, header = True)  


Epoch 1/16
31653/31653 [==============================] - 70s 2ms/step - loss: 0.5129 - acc: 0.8600
Epoch 2/16
   64/31653 [..............................] - ETA: 1:06 - loss: 0.5209 - acc: 0.8594

/usr/local/lib/python3.6/dist-packages/keras/callbacks.py:1109: RuntimeWarning: Reduce LR on plateau conditioned on metric `val_loss` which is not available. Available metrics are: loss,acc,lr
  (self.monitor, ','.join(list(logs.keys()))), RuntimeWarning


31653/31653 [==============================] - 68s 2ms/step - loss: 0.4108 - acc: 0.8769
Epoch 3/16
31653/31653 [==============================] - 68s 2ms/step - loss: 0.3832 - acc: 0.8814
Epoch 4/16
31653/31653 [==============================] - 68s 2ms/step - loss: 0.3691 - acc: 0.8840
Epoch 5/16
31653/31653 [==============================] - 68s 2ms/step - loss: 0.3593 - acc: 0.8868
Epoch 6/16
31653/31653 [==============================] - 68s 2ms/step - loss: 0.3560 - acc: 0.8878
Epoch 7/16
31653/31653 [==============================] - 68s 2ms/step - loss: 0.3552 - acc: 0.8874
Epoch 8/16
31653/31653 [==============================] - 68s 2ms/step - loss: 0.3447 - acc: 0.8894
Epoch 9/16
31653/31653 [==============================] - 68s 2ms/step - loss: 0.3413 - acc: 0.8903
Epoch 10/16
31653/31653 [==============================] - 67s 2ms/step - loss: 0.3411 - acc: 0.8912
Epoch 11/16
31653/31653 [==============================] - 67s 2ms/step - loss: 0.3486 - acc: 0.8883
Epoch 12/

/usr/local/lib/python3.6/dist-packages/keras/callbacks.py:1109: RuntimeWarning: Reduce LR on plateau conditioned on metric `val_loss` which is not available. Available metrics are: loss,acc,lr
  (self.monitor, ','.join(list(logs.keys()))), RuntimeWarning


31653/31653 [==============================] - 68s 2ms/step - loss: 0.4108 - acc: 0.8769
Epoch 3/16
31653/31653 [==============================] - 68s 2ms/step - loss: 0.3832 - acc: 0.8814
Epoch 4/16
31653/31653 [==============================] - 68s 2ms/step - loss: 0.3691 - acc: 0.8840
Epoch 5/16
31653/31653 [==============================] - 68s 2ms/step - loss: 0.3593 - acc: 0.8868
Epoch 6/16
31653/31653 [==============================] - 68s 2ms/step - loss: 0.3560 - acc: 0.8878
Epoch 7/16
31653/31653 [==============================] - 68s 2ms/step - loss: 0.3552 - acc: 0.8874
Epoch 8/16
31653/31653 [==============================] - 68s 2ms/step - loss: 0.3447 - acc: 0.8894
Epoch 9/16
31653/31653 [==============================] - 68s 2ms/step - loss: 0.3413 - acc: 0.8903
Epoch 10/16
31653/31653 [==============================] - 67s 2ms/step - loss: 0.3411 - acc: 0.8912
Epoch 11/16
31653/31653 [==============================] - 67s 2ms/step - loss: 0.3486 - acc: 0.8883
Epoch 12/

In [0]:
from keras.layers import CuDNNLSTM,CuDNNGRU
from keras.models import Sequential
from keras.layers import Dense, Flatten, Embedding, SimpleRNN, Conv1D, MaxPooling1D
from keras.layers import Dropout
from keras.layers import Dense
from keras import optimizers,callbacks
import keras
model4 = Sequential()
model4.add(Embedding(1000, 1000 ,input_length=25))


model4.add(CuDNNGRU(600, return_sequences = True))
model4.add(Dropout(0.7))

model4.add(CuDNNGRU(600))
model4.add(Dropout(0.6))

model4.add(Dense(100, activation='elu'))
model4.add(Dense(50, activation='relu'))
model4.add(Dense(8, activation='softmax'))

adam = optimizers.adam(lr = 0.001)
reduce_lr = callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=2)
model4.compile(optimizer=adam,
              loss='categorical_crossentropy',
              metrics=['acc'])
history = model4.fit(x_train, labelDataBinary,
                    epochs=16,
                    validation_split=0,
                    class_weight = class_weights,
                   callbacks=[reduce_lr])

model4.save('/content/m4.h5')
#0.87181 
predic4 = pd.Series(predict(model4, x_test))
submission = pd.concat([test.ID.astype(np.int), predic4], axis = 1)
submission.columns = ['ID' , 'MaterialType']
submission.to_csv('/content/submition_model4_add.1.csv', index = False, header = True)  


Epoch 1/16
31653/31653 [==============================] - 44s 1ms/step - loss: 0.4981 - acc: 0.8616
Epoch 2/16
   96/31653 [..............................] - ETA: 43s - loss: 0.4070 - acc: 0.8958

/usr/local/lib/python3.6/dist-packages/keras/callbacks.py:1109: RuntimeWarning: Reduce LR on plateau conditioned on metric `val_loss` which is not available. Available metrics are: loss,acc,lr
  (self.monitor, ','.join(list(logs.keys()))), RuntimeWarning


31653/31653 [==============================] - 42s 1ms/step - loss: 0.3960 - acc: 0.8815
Epoch 3/16
31653/31653 [==============================] - 43s 1ms/step - loss: 0.3708 - acc: 0.8853
Epoch 4/16
31653/31653 [==============================] - 43s 1ms/step - loss: 0.3513 - acc: 0.8911
Epoch 5/16
31653/31653 [==============================] - 42s 1ms/step - loss: 0.3364 - acc: 0.8939
Epoch 6/16
31653/31653 [==============================] - 42s 1ms/step - loss: 0.3260 - acc: 0.8953
Epoch 7/16
31653/31653 [==============================] - 42s 1ms/step - loss: 0.3155 - acc: 0.8984
Epoch 8/16
31653/31653 [==============================] - 42s 1ms/step - loss: 0.3064 - acc: 0.9003
Epoch 9/16
31653/31653 [==============================] - 42s 1ms/step - loss: 0.2979 - acc: 0.9029
Epoch 10/16
31653/31653 [==============================] - 43s 1ms/step - loss: 0.2934 - acc: 0.9038
Epoch 11/16
31653/31653 [==============================] - 42s 1ms/step - loss: 0.2882 - acc: 0.9049
Epoch 12/

**Using Embedding Layer**

In [0]:
"""from keras.layers import LSTM
from keras.models import Sequential
from keras.layers import Dense, Flatten, Embedding, SimpleRNN, Conv1D, MaxPooling1D
from keras.layers import Dropout
from keras.layers import Dense

model5 = Sequential()
model5.add(Embedding(1000, 300, input_length=25, weights=[embedding_matrix], trainable=False))

model5.add(Conv1D(64, 5, activation='relu'))
model5.add(MaxPooling1D(pool_size=4))


model5.add(LSTM(1000, recurrent_dropout = 0.3))
model5.add(Dropout(0.35))
model5.add(Dense(8, activation='softmax'))

model5.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['acc'])
history = model5.fit(x_train, labelDataBinary,
                    epochs=16,
                    batch_size=128,
                    validation_split=0.2,
                    class_weight = class_weights)


model5.save('/content/m5.h5')
predic5 = pd.Series(predict(model5, x_test))
submission = pd.concat([test.ID.astype(np.int), predic5], axis = 1)
submission.columns = ['ID' , 'MaterialType']
submission.to_csv('/content/submition_model5.csv', index = False, header = True)  
"""

In [0]:
from keras.layers import CuDNNLSTM,CuDNNGRU
from keras.models import Sequential
from keras.layers import Dense, Flatten, Embedding, SimpleRNN, Conv1D, MaxPooling1D
from keras.layers import Dropout
from keras.layers import Dense
from keras import optimizers,callbacks
import keras
model6 = Sequential()
model6.add(Embedding(1000, 1000 ,input_length=25))


model6.add(CuDNNGRU(600, return_sequences = True))
#model.add(Dropout(0.6))
model6.add(Dropout(0.7))

model6.add(CuDNNGRU(500))
model6.add(Dropout(0.7))

model6.add(Dense(100, activation='relu'))
model6.add(Dense(50, activation='elu'))
model6.add(Dense(8, activation='softmax'))

adam = optimizers.adam(lr = 0.001)
reduce_lr = callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=2)
model6.compile(optimizer=adam,
              loss='categorical_crossentropy',
              metrics=['acc'])
history = model6.fit(x_train, labelDataBinary,
                    epochs=16,
                    validation_split=0,
                    class_weight = class_weights,
                   callbacks=[reduce_lr])

#model6.save('/content/model_morefeatures.h5')
#v1_0.87181 
#0.87181 
#0.87695
predic6 = pd.Series(predict(model6, x_test))
submission = pd.concat([test.ID.astype(np.int), predic6], axis = 1)
submission.columns = ['ID' , 'MaterialType']
submission.to_csv('/content/submition_model6_add.1.csv', index = False, header = True)  


Epoch 1/16
31653/31653 [==============================] - 41s 1ms/step - loss: 0.4913 - acc: 0.8606
Epoch 2/16
   96/31653 [..............................] - ETA: 39s - loss: 0.4601 - acc: 0.8750

/usr/local/lib/python3.6/dist-packages/keras/callbacks.py:1109: RuntimeWarning: Reduce LR on plateau conditioned on metric `val_loss` which is not available. Available metrics are: loss,acc,lr
  (self.monitor, ','.join(list(logs.keys()))), RuntimeWarning


31653/31653 [==============================] - 40s 1ms/step - loss: 0.3912 - acc: 0.8815
Epoch 3/16
31653/31653 [==============================] - 40s 1ms/step - loss: 0.3625 - acc: 0.8866
Epoch 4/16
31653/31653 [==============================] - 40s 1ms/step - loss: 0.3479 - acc: 0.8907
Epoch 5/16
31653/31653 [==============================] - 40s 1ms/step - loss: 0.3342 - acc: 0.8929
Epoch 6/16
31653/31653 [==============================] - 40s 1ms/step - loss: 0.3219 - acc: 0.8961
Epoch 7/16
31653/31653 [==============================] - 40s 1ms/step - loss: 0.3149 - acc: 0.8972
Epoch 8/16
31653/31653 [==============================] - 40s 1ms/step - loss: 0.3063 - acc: 0.9012
Epoch 9/16
31653/31653 [==============================] - 42s 1ms/step - loss: 0.3031 - acc: 0.9009
Epoch 10/16
31653/31653 [==============================] - 41s 1ms/step - loss: 0.4913 - acc: 0.8606
Epoch 2/16
   96/31653 [..............................] - ETA: 39s - loss: 0.4601 - acc: 0.8750

/usr/local/lib/python3.6/dist-packages/keras/callbacks.py:1109: RuntimeWarning: Reduce LR on plateau conditioned on metric `val_loss` which is not available. Available metrics are: loss,acc,lr
  (self.monitor, ','.join(list(logs.keys()))), RuntimeWarning


31653/31653 [==============================] - 40s 1ms/step - loss: 0.3912 - acc: 0.8815
Epoch 3/16
31653/31653 [==============================] - 40s 1ms/step - loss: 0.3625 - acc: 0.8866
Epoch 4/16
31653/31653 [==============================] - 40s 1ms/step - loss: 0.3479 - acc: 0.8907
Epoch 5/16
31653/31653 [==============================] - 40s 1ms/step - loss: 0.3342 - acc: 0.8929
Epoch 6/16
31653/31653 [==============================] - 40s 1ms/step - loss: 0.3219 - acc: 0.8961
Epoch 7/16
31653/31653 [==============================] - 40s 1ms/step - loss: 0.3149 - acc: 0.8972
Epoch 8/16
31653/31653 [==============================] - 40s 1ms/step - loss: 0.3063 - acc: 0.9012
Epoch 9/16
31653/31653 [==============================] - 42s 1ms/step - loss: 0.3031 - acc: 0.9009
Epoch 10/16
31653/31653 [==============================] - 47s 1ms/step - loss: 0.2945 - acc: 0.9032
Epoch 11/16Epoch 11/16
31653/31653 [==============================] - 41s 1ms/step - loss: 0.2863 - acc: 0.905

In [0]:
from keras.layers import CuDNNLSTM,CuDNNGRU
from keras.models import Sequential
from keras.layers import Dense, Flatten, Embedding, SimpleRNN, Conv1D, MaxPooling1D
from keras.layers import Dropout
from keras.layers import Dense
from keras import optimizers,callbacks
import keras
model6 = Sequential()
model6.add(Embedding(1600, 1600 ,input_length=25))


model6.add(CuDNNGRU(600, return_sequences = True))
#model.add(Dropout(0.6))
model6.add(Dropout(0.7))

model6.add(CuDNNGRU(500))
model6.add(Dropout(0.6))

model6.add(Dense(100, activation='elu'))
model6.add(Dense(50, activation='elu'))
model6.add(Dense(8, activation='softmax'))

adam = optimizers.adam(lr = 0.0014)
reduce_lr = callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=2)
model6.compile(optimizer=adam,
              loss='categorical_crossentropy',
              metrics=['acc'])
history = model6.fit(x_train, labelDataBinary,
                    epochs=16,
                    validation_split=0,
                    class_weight = class_weights,
                   callbacks=[reduce_lr])

model6.save('/content/model_morefeatures.h5')
#v1_0.87181 
#0.87181 
#0.87695
predic6 = pd.Series(predict(model6, x_test))
submission = pd.concat([test.ID.astype(np.int), predic6], axis = 1)
submission.columns = ['ID' , 'MaterialType']
submission.to_csv('/content/submition_model_add_v1.csv', index = False, header = True)

Epoch 1/16
Epoch 1/16
31653/31653 [==============================] - 48s 2ms/step - loss: 0.4869 - acc: 0.8620
Epoch 2/16
   96/31653 [..............................] - ETA: 46s - loss: 0.3844 - acc: 0.8854

/usr/local/lib/python3.6/dist-packages/keras/callbacks.py:1109: RuntimeWarning: Reduce LR on plateau conditioned on metric `val_loss` which is not available. Available metrics are: loss,acc,lr
  (self.monitor, ','.join(list(logs.keys()))), RuntimeWarning
/usr/local/lib/python3.6/dist-packages/keras/callbacks.py:1109: RuntimeWarning: Reduce LR on plateau conditioned on metric `val_loss` which is not available. Available metrics are: loss,acc,lr
  (self.monitor, ','.join(list(logs.keys()))), RuntimeWarning


31653/31653 [==============================] - 47s 1ms/step - loss: 0.3938 - acc: 0.8806
Epoch 3/16
31653/31653 [==============================] - 46s 1ms/step - loss: 0.3687 - acc: 0.8845
Epoch 4/16Epoch 4/16
31653/31653 [==============================] - 46s 1ms/step - loss: 0.3640 - acc: 0.8866
Epoch 5/16
31653/31653 [==============================] - 47s 1ms/step - loss: 0.3506 - acc: 0.8890
Epoch 6/16
Epoch 6/16
31653/31653 [==============================] - 46s 1ms/step - loss: 0.3482 - acc: 0.8901
Epoch 7/16
Epoch 7/16
31653/31653 [==============================] - 46s 1ms/step - loss: 0.3426 - acc: 0.8921
Epoch 8/16
31653/31653 [==============================] - 46s 1ms/step - loss: 0.3418 - acc: 0.8927
Epoch 9/16
31653/31653 [==============================] - 47s 1ms/step - loss: 0.3384 - acc: 0.8929
Epoch 10/16
31653/31653 [==============================] - 46s 1ms/step - loss: 0.3381 - acc: 0.8949
Epoch 11/16
Epoch 11/16
31653/31653 [==============================] - 46s 1ms

In [0]:
submission.to_csv('/content/submition_model_impr.csv', index = False, header = True)  

In [0]:
"""
reverseLabelDict = {}

for key, value in labelDict.items():
  reverseLabelDict[value] = key
  
predictions = model.predict(x_test)
predictions = pd.Series([np.argmax(i) for i in predictions], index = test.index)
predictions = predictions.map(reverseLabelDict)

submission = pd.concat([test.ID.astype(np.int), predictions], axis = 1)
submission.columns = ['ID' , 'MaterialType']
"""

In [0]:
 # submission.to_csv('/content/submition_3.1_05_100_50_8.csv', index = False, header = True)

In [0]:
""""reverseLabelDict = {}

for key, value in labelDict.items():
  reverseLabelDict[value] = key
def predict(model, x_test):
  predictions = model.predict(x_test)
  predictions = pd.Series([np.argmax(i) for i in predictions], index = test.index)
  predictions = predictions.map(reverseLabelDict)
  submission = pd.concat([test.ID.astype(np.int), predictions], axis = 1)
  submission.columns = ['ID' , 'MaterialType']
  return predictions"""

In [0]:
num_model = 6
predic1 = pd.Series(predict(model1, x_test))
predic2 = pd.Series(predict(model2, x_test))
predic3 = pd.Series(predict(model3, x_test))
predic4 = pd.Series(predict(model4, x_test))
predic5 = pd.Series(predict(model5, x_test))
predic6 = pd.Series(predict(model6, x_test))

In [0]:
"""total = pd.DataFrame(pd.concat([predic1, predic2, predic3, predic4, predic5,predic6], axis = 1))
submissions = total.mode(axis = 1)[0]
submission = pd.concat([test.ID.astype(np.int), predictions], axis = 1)
submission.columns = ['ID' , 'MaterialType']
submission.to_csv('sub-agg.csv', index = False, header = True)"""

In [0]:
total = pd.DataFrame(pd.concat([predic1, predic2, predic3, predic4, predic5,predic6], axis = 1))
submissions = total.mode(axis = 1)[0]
#submissions = pd.concat([test.ID.astype(np.int), predictions], axis = 1)
submissions.columns = ['ID' , 'MaterialType']
submissions.to_csv('/content/sub-final.csv', index = False, header = True)